# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

#### Задание

1. Загрузите датасет.

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

    "FreeMsg: Txt: claim your reward of 3 hours talk time"

    "Have you visited the last lecture on physics?"

    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

    "Only 99$"

    Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

    По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [90]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [3]:
# сохраним ответ в блокнот
def save_answers(k, s):
    with open(s, "w") as fout:
        fout.write(str(k))

In [4]:
data = pd.read_csv('SMSSpamCollection.txt', names=['Spam','Col'], sep='\t', lineterminator='\n')

In [5]:
data.head()

,Spam,Col
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data.Spam.value_counts()

ham     4825
spam     747
Name: Spam, dtype: int64

In [7]:
X = list(data.Col)
Y = list(data.Spam)

In [8]:
# замена ham и spam на бинарные значения
mapping = {'ham':0, 'spam':1}
Y = [mapping[a] for a in Y]

Оценка качества логистической регрессии по признакам счетчика частот слов

In [9]:
print cross_val_score(Pipeline([("vectorizer", CountVectorizer()),
                                ("classifier", LogisticRegression(penalty='l1', random_state=2))]), X, Y, cv=10).mean()

0.979718496587


In [10]:
save_answers(0.9,'1.txt')

In [11]:
# обучение модели
clf = Pipeline([("vectorizer", CountVectorizer()), ("classifier", LogisticRegression(penalty='l1', random_state=2))])
clf.fit(X,Y)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...nalty='l1', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Проверка данной модели на новых данных

In [36]:
S=[]
S.append("FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB")
S.append("FreeMsg: Txt: claim your reward of 3 hours talk time")
S.append("Have you visited the last lecture on physics?")
S.append("Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$")
S.append("Only 99$")

In [38]:
for s in S:
    print clf.predict([s])

[1]
[1]
[0]
[0]
[0]


In [39]:
save_answers("1 1 0 0 0",'2.txt')

Изменение n-грамм

In [49]:
# только Биграммы ngram_range=(2,2)
print cross_val_score(Pipeline([("vectorizer", CountVectorizer(ngram_range=(2,2))),
                            ("classifier", LogisticRegression(penalty='l1', random_state=2))]), X, Y, cv=10, scoring='f1').mean()

0.835232617107


In [50]:
# только Триграммы: ngram_range=(3,3)
print cross_val_score(Pipeline([("vectorizer", CountVectorizer(ngram_range=(3,3))),
                            ("classifier", LogisticRegression(penalty='l1', random_state=2))]), X, Y, cv=10, scoring='f1').mean()

0.705367843155


In [51]:
# все: Униграммы, Биграммы, Триграммы: ngram_range=(1,3)
print cross_val_score(Pipeline([("vectorizer", CountVectorizer(ngram_range=(1,3))),
                            ("classifier", LogisticRegression(penalty='l1', random_state=2))]), X, Y, cv=10, scoring='f1').mean()

0.922413748293


In [52]:
save_answers("0.84 0.71 0.92",'3.txt')

Изменение n-грамм с Наивным Байесом (MultinomialNB) Посчитаем без Pipeline (с ним ошибка)

In [85]:
vectorizer = CountVectorizer(ngram_range=(2,2))
matrix = vectorizer.fit_transform(X)

cross_val_score(MultinomialNB(), matrix, Y, cv=10, scoring='f1').mean()

0.64550151779854426

In [86]:
vectorizer = CountVectorizer(ngram_range=(3,3))
matrix = vectorizer.fit_transform(X)

cross_val_score(MultinomialNB(), matrix, Y, cv=10, scoring='f1').mean()

0.37871948524573595

In [87]:
vectorizer = CountVectorizer(ngram_range=(1,3))
matrix = vectorizer.fit_transform(X)

cross_val_score(MultinomialNB(), matrix, Y, cv=10, scoring='f1').mean()

0.88848596560610016

In [88]:
save_answers("0.65 0.38 0.89",'4.txt')

Используем вместе со счетчиком частот слов TF-IDF (странно но ручной результат и результат с Pipeline отличаются)

In [109]:
transformer_v = TfidfVectorizer()
matrix_v = transformer_v.fit_transform(X)

cross_val_score(LogisticRegression(penalty='l1', random_state=2), matrix, Y, cv=10, scoring='f1').mean()

0.92096276914379227

In [113]:
print cross_val_score(Pipeline([("vectorizer", TfidfVectorizer()),
                            ("classifier", LogisticRegression(penalty='l1', random_state=2))]), X, Y, cv=10, scoring='f1').mean()

0.875145675867


In [111]:
# в любом случае качество понизилось
save_answers(-1,'5.txt')